In [ ]:
import pandas as pd
import json
from collections import defaultdict
import re
import datetime, time

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados

In [ ]:
def extraerTiempos(rawData):
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erAccessed = re.compile(r'\baccessed$\b')
    erInteracted = re.compile(r'\binteracted$\b')

    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    erSeriousGame = re.compile(r'\bserious-game$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erScreen = re.compile(r'\bscreen$\b')

    erExitGamePanel = re.compile(r'\bexit_game_panel$\b')
    erExitGamePanelCloseButton = re.compile(r'\bexit_game_panel_close_button$\b')
    
    data = defaultdict(defaultdict)
    inicioYFinJuego = defaultdict(list)
    anomalias = []
    
    for evento in rawData:         
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]

        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in data[name]:
                        data[name][levelCode].append({"ini" : timestamp, "fin" : "", "stars" : ""})
                    else:
                        data[name][levelCode] = [{"ini" : timestamp, "fin" : "", "stars" : ""}]
                #else: #-> sería cuando un nivel es reiniciado, por ahora no vamos a tenerlo en cuenta
            elif erCompleted.search(verb):
                if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = timestamp
                        data[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
                else:
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = None #Porque el jugador sale al menú o cierra el juego
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
        
        #Algunos jugadores cierran el juego el juego con Alt+F4 o dando a la x de la ventana, esto no queda registrado
        elif erSeriousGame.search(obj) and erInitialized.search(verb): #Nos indica que el jugador ha abierto el juego
            inicioYFinJuego[name].append({"inicio" : timestamp, "fin" : None})
            
        elif erAccessed.search(verb) and erScreen.search(obj) and erExitGamePanel.search(objectId): #El jugador ha abierto el panel para cerrar el juego
            inicioYFinJuego[name][-1]["fin"] = timestamp
           
        elif erInteracted.search(verb) and erGameObject.search(obj) and erExitGamePanelCloseButton.search(objectId): #El jugador ha cerrado el panel de cerrar juego, por lo tanto no ha terminado
            inicioYFinJuego[name][-1]["fin"] = timestamp
            
    return {"tiemposNivel" : data, "tiempoTotal" : inicioYFinJuego, "anomalias" : anomalias}              

In [ ]:
#Esta clase sirve para parsear timestamps, recibe 2 y hace la diferencia
class Tiempo:
    def __init__ (self, inicio, final = -1, seconds = -1): #Necesario para tener 2 constructores
        if final == -1: #Primer constructor (1 argumento)
            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            erHours = re.compile(r'\d*h')
            erMinutes = re.compile(r'\d*m')
            erSeconds = re.compile(r'\d*s')
            h = erHours.search(inicio)
            m = erMinutes.search(inicio)
            s = erSeconds.search(inicio)

            if h != None:
                self.hours = h[0][:-1]
            if m != None:
                self.minutes = m[0][:-1]
            if s != None:
                self.seconds = s[0][:-1]

        elif seconds == -1: #Segundo constructor (2 aregumentos)
            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            fin = time.mktime(datetime.datetime.strptime(final, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())
            ini = time.mktime(datetime.datetime.strptime(inicio, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())

            hoursRaw = int((fin - ini)/3600)
            minutesRaw = int((fin - ini)/60)
            secondsRaw = int(fin - ini)

            if hoursRaw != 0:
                self.hours = hoursRaw
                minutesRaw = int(((fin - ini)%3600)/60)
            if minutesRaw != 0:
                self.minutes = minutesRaw
                secondsRaw = int((fin - ini)%60)
            self.seconds = secondsRaw
            
        else: #Tercer constructor (3 argumentos)
            self.hours = inicio
            self.minutes = final
            self.seconds = seconds
            
        self.hours = int(self.hours)
        self.minutes = int(self.minutes)
        self.seconds = int(self.seconds)
            
    def toString(self):
        return self.__repr__()
    
    def segundos(self):
        return self.hours*3600 + self.minutes*60 + self.seconds

    def __repr__(self):
        t=""
        if self.hours !=0:
            t += str(self.hours) + "h/"
        if self.minutes !=0:
            t += str(self.minutes) + "m/"
        if self.seconds !=0:
            t += str(self.seconds) + "s"
        else:
            t = t[:-1]
        return t
    
    def __add__(self, t): #Sobrecarga del operador +
        s = self.seconds + t.seconds
        m = int(self.minutes) + t.minutes
        h = self.hours + t.hours
        
        if s >= 60:
            s -= 60
            m += 1
        if m >= 60:
            m -= 60
            h += 1
            
        return Tiempo(h,m,s)
    
    def __mul__(self, n): #Sobrecarga del operador *
        s = self.seconds * n
        m = self.minutes * n
        h = self.hours * n
        
        m += (h - int(h))*60
        s += (m - int(m))*60
        
        if s >= 60:
            m += int(s/60)
            s = s%60
        if m >= 60:
            h += int(m/60)
            m = m%60
        
        return Tiempo(h,m,s)
    
    def __truediv__(self, n): #Sobrecarga del operador /
        selfS = self.seconds
        selfM = self.minutes
        selfH = self.hours
        
        h = int(selfH/n)
        selfM += (selfH%n)*60
        m = int(selfM/n)
        selfS += (selfM%n)*60
        s = int(selfS/n)
        
        if s >= 60:
            m += int(s/60)
            s = s%60
        if m >= 60:
            h += int(m/60)
            m = m%60
        return Tiempo(int(h),int(m),int(s))
    
    def __le__(self, t):     
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        sT = t.hours*3600 + t.minutes*60 + t.seconds
        
        return sSelf <= sT
    
    def __ge__(self, t):
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        sT = t.hours*3600 + t.minutes*60 + t.seconds
        
        return sSelf >= sT
    
    def __lt__(self, t):     
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        sT = t.hours*3600 + t.minutes*60 + t.seconds
        
        return sSelf < sT
    
    def __gt__(self, t):
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        sT = t.hours*3600 + t.minutes*60 + t.seconds
        
        return sSelf > sT

In [ ]:
def tiempoPorNiveles(data, anomalias):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    try:
                        timeDifference = Tiempo(times["ini"], times["fin"])
                        if level in tiemposJugados[player]:
                            tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                        else:
                            tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                    except ValueError: #Algunos timestamps de finalizacion estan vacios, ¿Tiene que ser asi?
                        anomalias.append({"jugador" : player,
                                  "nivel" : level,
                                  "Timestamp-Inicio" : times["ini"],
                                  "Timestamp-Fin" : times["fin"],
                                  "Descripcion:" : "No se ha podido parsear uno de los 2 timestamps"})
    return tiemposJugados

In [ ]:
def buscarTarjetas(rawData):
    data = []
    erIdLevel = re.compile(r'/')
    for evento in rawData:
        idJugador = evento["actor"]["name"]
        levelCode = erIdLevel.split(evento["object"]["id"])[-1]
        if idJugador == "bdwyj" and "result" in evento and "extensions" in evento["result"] and "level" in evento["result"]["extensions"]:
            if evento["result"]["extensions"]["level"] == "variables_9":
                data.append(evento)
        elif idJugador == "bdwyj" and levelCode == "variables_9":
            data.append(evento)
    return data

In [ ]:
def getMediaTiemposYEstrellas(data):
    tiemposYEstrellasMedios = defaultdict()
    sumasTotales = defaultdict()
    for user in data:
        for level in data[user]:
            for time in data[user][level]:
                if level in tiemposYEstrellasMedios:
                    tiemposYEstrellasMedios[level]["tiempo"] += time["time"]
                    tiemposYEstrellasMedios[level]["stars"] += time["stars"]
                    sumasTotales[level] += 1
                else:
                    tiemposYEstrellasMedios[level] = {"tiempo" : time["time"], "stars" : time["stars"]}
                    sumasTotales[level] = 1
    for level in tiemposYEstrellasMedios:
        tiemposYEstrellasMedios[level]["tiempo"] = tiemposYEstrellasMedios[level]["tiempo"]/sumasTotales[level];
        tiemposYEstrellasMedios[level]["stars"] = round(tiemposYEstrellasMedios[level]["stars"]/sumasTotales[level], 2);
    return tiemposYEstrellasMedios

In [ ]:
def getDesempenioJugadores(tiempos, tiemposMedios, porcDebajo, porcEncima, primerIntento = True):
    #tiempos -> diccionario con los tiempos de cada jugador en cada nivel
    #tiemporMedios -> media de los tiempos por nivel
    #porcDebajo -> porcentaje que se usara para filtrar a los jugadores con un tiempo igual o inferior a este
    #porcEncima -> porcentaje que se usara para filtrar a los jugadores con un tiempo igual o superior a este
    #primerIntento -> true cuando solo se quiere tener en cuenta el primer intento del jugador, false cuando no
    jugadoresPorDebajo = defaultdict(defaultdict) #Jugadores con buen desempeño, menor tiempo que media
    jugadoresPorEncima = defaultdict(defaultdict) #Jugaodre con mal desempeño, mayor tiempo que media

    for player in tiempos:
        for level in tiempos[player]:
            for time in tiempos[player][level]:
                if time["time"] <= (tiemposMedios[level]["tiempo"]*(1-porcDebajo)):
                    if level in jugadoresPorDebajo[player]:
                        jugadoresPorDebajo[player][level].append(time["time"])
                    else:
                        jugadoresPorDebajo[player][level] = [time["time"]]
                        
                elif time["time"] >= (tiemposMedios[level]["tiempo"]*(1+porcEncima)):
                    if level in jugadoresPorEncima[player]:
                        jugadoresPorEncima[player][level].append(time["time"])
                    else:
                        jugadoresPorEncima[player][level] = [time["time"]]
                        
                if primerIntento:
                    break
                    
    return jugadoresPorDebajo, jugadoresPorEncima

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [ ]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
result = extraerTiempos(rawData)
JSONFile.close()
#pd.DataFrame(result["data"])
pd.DataFrame.from_dict(result["tiempoTotal"], orient='index').transpose()

In [ ]:
tiemposJ = tiempoPorNiveles(result["tiemposNivel"], result["anomalias"])
#pd.DataFrame(tiempos)

In [ ]:
tiemposMedios = getMediaTiemposYEstrellas(tiemposJ)
#pd.DataFrame(tiemposMedios)

In [ ]:
porcentajePorDebajo = 0.2
porcentajePorEncima = 0.7
jPorDebajo, jPorEncima = getDesempenioJugadores(tiemposJ, tiemposMedios, porcentajePorDebajo, porcentajePorEncima)
pd.DataFrame(jPorDebajo)